# Automatisierte Erstellung der Files

In [29]:
import requests
import os
import json
import pandas as pd
import geopandas as gpd
import networkx as nx
import osmnx as ox
import numpy as np
from shapely.geometry import Point, LineString, Polygon
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt


In [54]:
path = "Daten/Move-Files"
path_Rosengarten = "Daten/Move-Files/jobs_3068759_results_Rosengartenstrasse.json"
path_Wasserwerk = "Daten/Move-Files/jobs_3074055_results_Wasserwerkstrasse.json"
path_Albisrieden = "Daten/Move-Files/jobs_3074056_results_Albisrieden_.json"


In [55]:
#for filename in os.listdir("Daten/Move-Files"):
#    if filename.endswith('.json'):
#        with open(os.path.join("Daten/Move-Files", filename)) as f:
#            data = f.read()

In [56]:
with open(path_Rosengarten) as f:
    dataRosengarten = json.load(f)

In [57]:
with open(path_Wasserwerk) as f:
    dataWasserwerk = json.load(f)

In [58]:
with open(path_Albisrieden) as f:
    dataAlbisrieden = json.load(f)

In [59]:
datas = [dataRosengarten, dataWasserwerk, dataAlbisrieden]

In [60]:
PERCENTILESIZE = 5 #Konstante (Variable)

def getSpeedPercentage(percentileValues, minSpeed, maxSpeed): #Geschwindigkeitsperzentil, min Temp
    minCounter = 0
    maxCounter = 0
    for percentileValue in percentileValues:
        if percentileValue <= minSpeed:
            minCounter = minCounter + 1
        if percentileValue < maxSpeed:
            maxCounter = maxCounter + 1
    return (maxCounter - minCounter) * PERCENTILESIZE

In [61]:
for data in datas:
    for route in data["routes"]:
        for segment in route["segmentResults"]:
            speedLimit = segment["speedLimit"]
            for timeResult in segment["segmentTimeResults"]:
                timeResult["overspeed"] = [{
                    "percentage": getSpeedPercentage(timeResult["speedPercentiles"], speedLimit + 0, speedLimit + 5),
                    "min": speedLimit + 0,
                    "max": speedLimit + 5},
                    {"percentage": getSpeedPercentage(timeResult["speedPercentiles"], speedLimit + 6, speedLimit + 10),
                    "min": speedLimit + 6,
                    "max": speedLimit + 10},
                    {"percentage": getSpeedPercentage(timeResult["speedPercentiles"], speedLimit + 11, speedLimit + 150),
                    "min": speedLimit + 10,
                    "max": speedLimit + 150}]

In [50]:
NightSegments = []
MorningSegments = []
AfternoonSegments = []
EveningSegments = []

def getRouteSegmentData(route):
    RouteData = {
        "routeName": route["routeName"],
        "timeSets": {}
    }
    for segment in route["segmentResults"]:
        street = segment["streetName"] if "streetName" in segment else ""
        medianSpeed = segment["medianSpeed"] if "medianSpeed" in segment else ""
        speedLimit = segment["speedLimit"]
        shape = segment["shape"]
        for timeResult in segment["segmentTimeResults"]:
            NewSegment = {
                 "routeName": route["routeName"], 
                 "street": street,
                 "speedLimit": speedLimit,
                 "shape": shape, 
                 "innerhalb Toleranz-Abzug": timeResult["overspeed"][0]["percentage"],
                 "min 5 km/h zu schnell": timeResult["overspeed"][1]["percentage"],
                 "min 10 km/h zu schnell": timeResult["overspeed"][2]["percentage"],
                 "insgesamt zu schnell": timeResult["overspeed"][0]["percentage"] + timeResult["overspeed"][1]["percentage"] + timeResult["overspeed"][2]["percentage"]
            }
            
            if timeResult["timeSet"] not in RouteData["timeSets"]:
                RouteData["timeSets"][timeResult["timeSet"]] = []
            RouteData["timeSets"][timeResult["timeSet"]].append(NewSegment)


    return RouteData

In [51]:
aWay = getRouteSegmentData(data["routes"][0])
bWay = getRouteSegmentData(data["routes"][1])

In [64]:
def coordinatesToPoints(dataFrame):
    for index, entry in dataFrame.iterrows():
        Points = []
        Line = []
        for coordinate in entry["shape"]:
            lat = coordinate["latitude"]
            long = coordinate["longitude"]
            newPoint = Point(long, lat)
            Points.append(newPoint)
        Line = LineString(Points)
        dataFrame.at[index,"geometry"] = Line
    NewFrame = dataFrame.drop(['shape'], axis=1)
    return NewFrame

In [69]:
OUTPATH="Daten/out"
routeDatas={}

for data in datas:
    for route in data["routes"]:
        routeData = getRouteSegmentData(route)
        routeData["gdfs"] = {}
        for key, timeSet in routeData["timeSets"].items():
            dataFrame = pd.DataFrame(timeSet)
            pointDf = coordinatesToPoints(dataFrame)
            gdf = gpd.GeoDataFrame(pointDf, geometry="geometry")
            gdf = gdf.set_crs(epsg=4326)
            routeData["gdfs"][key] = gdf
            gdf.to_file( OUTPAHT + "/" + routeData["routeName"] + "_" + str(key) + ".geojson", driver="GeoJSON") 
        routeDatas[routeData["routeName"]] = routeData
